In [5]:
# PROXY
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
from datasets import load_dataset
datas = load_dataset("bigbio/biored")

/root/miniconda3/lib/python3.8/site-packages/datasets/load.py:1461: FutureWarning: The repository for bigbio/biored contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/biored
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


In [7]:
from text2knowledge.strategy1 import extract_concepts

import json

In [8]:
model_name = 'mistral:latest'

In [29]:
datas['test'][2]

{'pmid': '18457324',
 'passages': [{'type': 'title',
   'text': ['Genetic polymorphisms in the carbonyl reductase 3 gene CBR3 and the NAD(P)H:quinone oxidoreductase 1 gene NQO1 in patients who developed anthracycline-related congestive heart failure after childhood cancer.'],
   'offsets': [[0, 207]]},
  {'type': 'abstract',
   'text': ['BACKGROUND: Exposure to anthracyclines as part of cancer therapy has been associated with the development of congestive heart failure (CHF). The potential role of genetic risk factors in anthracycline-related CHF remains to be defined. Thus, in this study, the authors examined whether common polymorphisms in candidate genes involved in the pharmacodynamics of anthracyclines (in particular, the nicotinamide adenine dinucleotide phosphate:quinone oxidoreductase 1 gene NQO1 and the carbonyl reductase 3 gene CBR3) had an impact on the risk of anthracycline-related CHF. METHODS: A nested case-control study was conducted within a cohort of 1979 patients enro

In [35]:
import os, sys
import contextlib

i = 4
while i < len(datas['test']):
    data = datas['test'][i]
    pmid = data['pmid']
    passages = data['passages']
    entities = data['entities']
    abstract = passages[1]['text'][0]
    abstract_offset = passages[1]['offsets'][0]
    metadata = {
        'pmid': pmid,
        'type': 'abstract',
    }

    #with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    entities_extracted = extract_concepts(abstract, model=model_name, metadata=metadata)
    
    if entities_extracted:
        disease_entities_extracted = [e for e in entities_extracted if e['category'] == 'Disease']
        # print(disease_entities_extracted)
        entities_str = json.dumps(entities_extracted, indent=4)
        # print(entities_str)
    else:
        print("No entities found.")
    break
    # i += 1
    # if i >= 2:
    #     break

 [
    {
        "entity": "deoxyguanosine kinase (dGK)",
        "confidence": "5",
        "category": "Enzyme"
    },
    {
        "entity": "DGUOK gene",
        "confidence": "5",
        "category": "Gene"
    },
    {
        "entity": "mitochondrial DNA depletion",
        "confidence": "4",
        "category": "MetabolicProcess"
    },
    {
        "entity": "hepatocerebral phenotype",
        "confidence": "3",
        "category": "Symptom"
    },
    {
        "entity": "combined respiratory chain deficiencies",
        "confidence": "4",
        "category": "MetabolicProcess"
    },
    {
        "entity": "c.444-62C>A",
        "confidence": "5",
        "category": "Mutation"
    },
    {
        "entity": "dGK kinase domain",
        "confidence": "3",
        "category": "ProteinDomain"
    },
    {
        "entity": "North-African population",
        "confidence": "3",
        "category": "Population"
    },
    {
        "entity": "subtentorial abnormal myelination

In [10]:
entities

[{'text': ['PKC-alpha'],
  'offsets': [[11, 20]],
  'concept_id': '18750',
  'semantic_type_id': 'GeneOrGeneProduct'},
 {'text': ['lithium'],
  'offsets': [[32, 39]],
  'concept_id': 'D008094',
  'semantic_type_id': 'ChemicalEntity'},
 {'text': ['nephrogenic diabetes insipidus'],
  'offsets': [[48, 78]],
  'concept_id': 'D018500',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['Lithium'],
  'offsets': [[80, 87]],
  'concept_id': 'D008094',
  'semantic_type_id': 'ChemicalEntity'},
 {'text': ['nephrogenic diabetes insipidus'],
  'offsets': [[125, 155]],
  'concept_id': 'D018500',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['NDI'],
  'offsets': [[157, 160]],
  'concept_id': 'D018500',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['patients'],
  'offsets': [[173, 181]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['lithium'],
  'offsets': [[244, 251]],
  'concept_id': 'D008094',
  'semantic_type_id': 'Chem

In [15]:
entities_extracted

In [16]:
disease_entities_extracted = [e for e in entities_extracted if e['category'] == 'Disease']
disease_entities_extracted

[{'entity': 'nephrogenic diabetes insipidus (NDI)',
  'confidence': '5',
  'category': 'Disease',
  'pmid': '25006961',
  'type': 'abstract'}]

In [8]:
abstract_offset

[122, 1815]

In [9]:
abstract

'OBJECTIVE: Congenital long QT syndrome (LQTS) with in utero onset of the rhythm disturbances is associated with a poor prognosis. In this study we investigated a newborn patient with fetal bradycardia, 2:1 atrioventricular block and ventricular tachycardia soon after birth. METHODS: Mutational analysis and DNA sequencing were conducted in a newborn. The 2:1 atrioventricular block improved to 1:1 conduction only after intravenous lidocaine infusion or a high dose of mexiletine, which also controlled the ventricular tachycardia. RESULTS: A novel, spontaneous LQTS-3 mutation was identified in the transmembrane segment 6 of domain IV of the Na(v)1.5 cardiac sodium channel, with a G-->A substitution at codon 1763, which changed a valine (GTG) to a methionine (ATG). The proband was heterozygous but the mutation was absent in the parents and the sister. Expression of this mutant channel in tsA201 mammalian cells by site-directed mutagenesis revealed a persistent tetrodotoxin-sensitive but li

In [12]:
def highlight_entities_label(s, start_idx, entities, color='#66CC66'):
    left = f"<span style='background-color: {color}'>"
    right = "</span>"
    final = ""
    last_r = 0
    for entity in entities:
        l, r = entity['offsets'][0]
        if not l >= start_idx:
            continue
        l -= start_idx
        r -= start_idx
        final += s[last_r:l]
        final += left + s[l:r] + right
        last_r = r
    final += s[r:]
    return final

In [13]:
from IPython.display import HTML

label = highlight_entities_label(abstract, abstract_offset[0], entities)
HTML(label)

In [14]:
def highlight_entities_extracted(s, entities, color='#00CCFF'):
    import re
    left = f"<span style='background-color: {color}'>"
    right = "</span>"
    final = s[:]
    for entity in entities:
        entity_text = entity['entity']
        final = re.sub('(?i)'+re.escape(entity_text), lambda m: left+entity_text+right, final)
    return final
predicted = highlight_entities_extracted(abstract, entities_extracted)
HTML(predicted)

TypeError: 'NoneType' object is not iterable

In [25]:
entities_extracted

[{'entity': 'Congenital long QT syndrome (LQTS)',
  'importance': 5,
  'category': 'Disease',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'In utero onset of rhythm disturbances',
  'importance': 3,
  'category': 'BiologicalProcess',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Newborn patient',
  'importance': 4,
  'category': 'Anatomy',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Fetal bradycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': '2:1 atrioventricular block',
  'importance': 4,
  'category': 'Pathway',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Ventricular tachycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Mutational analysis and DNA sequencing',
  'importance': 4,
  'category': 'MolecularFunction',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Lidocaine infusion',
  'importance': 3,
  

In [21]:
import re

In [28]:
confusion_matrix = {"test":[0,0,0]}
categories = []

def match_entity(entity_a, entity_b):
    return entity_a['category'] == entity_b['category'] and\
model.query(entity_a['entity'], top_k=1)[2] == model.query(entity_b['entity'], top_k=1)[2]

def macro_avg_f1_score(ans, gt):
    # for sample_ans, sample_gt in zip(ans, gt):
        # for category in categories:
            # get all entities in the category
    # ans_entities = [entity['entity']
    #                 for entity in sample_ans if entity['category'] == category]
    # ans_entities = [entity['entity'] for entity in ans]
    # # gt_entities = [entity['text']
    # #                for entity in sample_gt if entity['category'] == category]
    # gt_entities = [entity['text'][0] for entity in gt]
    # TP = len(set(ans_entities) & set(gt_entities))
    # FP = len(set(ans_entities) - set(gt_entities))
    # FN = len(set(gt_entities) - set(ans_entities))
    # confusion_matrix["test"][0] += TP
    # confusion_matrix["test"][1] += FP
    # confusion_matrix["test"][2] += FN
    # f1_score = []
    # # for category in categories:
    # TP, FP, FN = confusion_matrix["test"]
    # precision = TP/(TP+FP) if TP+FP else float('nan')
    # recall = TP/(TP+FN) if TP+FN else float('nan')
    # f1_score.append(2*(precision * recall) / (precision + recall))
    # return sum(f1_score) / len(f1_score)

macro_avg_f1_score(entities_extracted, entities)

In [44]:
entities_extracted

[{'entity': 'Congenital long QT syndrome (LQTS)',
  'importance': 5,
  'category': 'Disease',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'In utero onset of rhythm disturbances',
  'importance': 3,
  'category': 'BiologicalProcess',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Newborn patient',
  'importance': 4,
  'category': 'Anatomy',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Fetal bradycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': '2:1 atrioventricular block',
  'importance': 4,
  'category': 'Pathway',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Ventricular tachycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Mutational analysis and DNA sequencing',
  'importance': 4,
  'category': 'MolecularFunction',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Lidocaine infusion',
  'importance': 3,
  

In [10]:
entities[1]

{'text': ['long QT syndrome'],
 'offsets': [[56, 72]],
 'concept_id': 'D008133',
 'semantic_type_id': 'DiseaseOrPhenotypicFeature'}

In [11]:
entities_extracted[0]

{'entity': 'Congenital long QT syndrome (LQTS)',
 'importance': 5,
 'category': 'Disease',
 'pmid': '15485686',
 'type': 'abstract'}

In [10]:
set([j['semantic_type_id'] for i in datas["test"] for j in i['entities']])

{'CellLine',
 'ChemicalEntity',
 'DiseaseOrPhenotypicFeature',
 'GeneOrGeneProduct',
 'OrganismTaxon',
 'SequenceVariant'}

In [2]:
from text2knowledge.normalizer import Normalizer

In [4]:
class Arg:
    def __init__(self, category):
        self.show_embeddings = False
        self.show_predictions = True
        self.use_cuda = True
        if category == 'Gene':
            self.model_name_or_path = "dmis-lab/biosyn-biobert-bc2gn"
            # self.dictionary_path = "/root/text2knowledge/data/dictionary/dict_Gene.txt"
        self.dictionary_path = f"/root/text2knowledge/data/dictionary/yang/dict_{category}.txt"
        
            
            
arg_mf = Arg('MolecularFunction')
mf_normalizer = Normalizer(arg_disease)
mf_normalizer.normalize("G-->A substitution at codon 1763")

Loaded dictionary from cached file ./tmp/cached_biosyn-biobert-bc2gn_dict_Gene.pk


{'mention': 'g a substitution at codon 1763',
 'predictions': [{'name': 'transfer RNA isoleucine 17 (anticodon AAU)',
   'id': 'EntrezGene:100189310'},
  {'name': 'transfer RNA isoleucine (anticodon AAU)',
   'id': 'EntrezGene:107986683|107986679'},
  {'name': 'tRNA glutamic acid (anticodon UUC)', 'id': 'EntrezGene:7193'},
  {'name': 'tRNA glycine1 (anticodon CCC)', 'id': 'EntrezGene:7195'},
  {'name': 'tRNA glutamine 1 (anticodon CUG)', 'id': 'EntrezGene:7228'}]}

In [7]:
normalizer.biosyn.embed_sparse(names=["G-->A substitution at codon 1763"])

TypeError: 'NoneType' object is not callable

In [2]:
from text2knowledge.normalizer import normalize

class Arg:
    def __init__(self):
        self.mention = "long QT syndrome (LQTS)"
        self.model_name_or_path = "dmis-lab/biosyn-biobert-ncbi-disease"
        self.show_embeddings = False
        self.show_predictions = True
        self.dictionary_path = "/root/text2knowledge/data/dictionary/dict_Disease_20210630.txt"
        self.use_cuda = True
arg = Arg()
normalize(arg)



KeyboardInterrupt



In [6]:
from text2knowledge.normalizer import Normalizer

In [7]:
class Arg:
    def __init__(self):
        # self.mention = "long QT syndrome (LQTS)"
        self.model_name_or_path = "dmis-lab/biosyn-biobert-ncbi-disease"
        self.show_embeddings = False
        self.show_predictions = True
        self.dictionary_path = "/root/text2knowledge/data/dictionary/dict_Disease_20210630.txt"
        self.use_cuda = True
arg = Arg()

disease_normalizer = Normalizer(arg)

/root/miniconda3/lib/python3.8/site-packages/huggingface_hub/file_download.py:678: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainabi

In [8]:
disease_normalizer.normalize("long QT syndrome (LQTS)")

/root/miniconda3/lib/python3.8/site-packages/biosyn-0.1-py3.8.egg/biosyn/biosyn.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Loaded dictionary from cached file ./tmp/cached_biosyn-biobert-ncbi-disease_dict_Disease_20210630.pk


{'mention': 'long qt syndrome lqts',
 'predictions': [{'name': 'Long QT Syndrome', 'id': 'MESH:D008133'},
  {'name': 'Long Qt Syndrome 1/2', 'id': 'MESH:C566006'},
  {'name': 'Long Qt Syndrome 2|LONG QT SYNDROME 1/2, DIGENIC, INCLUDED;LQT1/2, DIGENIC, INCLUDED|LONG QT SYNDROME 2/3, DIGENIC, INCLUDED;LQT2/3, DIGENIC, INCLUDED|LONG QT SYNDROME 2/5, DIGENIC, INCLUDED;LQT2/5, DIGENIC, INCLUDED|LONG QT SYNDROME 2/9, DIGENIC, INCLUDED;LQT2/9, DIGENIC, INCLUDED|LONG QT SYNDROME 2, ACQUIRED, SUSCEPTIBILITY TO, INCLUDED|LQT2',
   'id': 'MESH:C563614'},
  {'name': 'Long Qt Syndrome 10', 'id': 'OMIM:611819,MESH:C567514'},
  {'name': 'LONG QT SYNDROME 13', 'id': 'OMIM:613485'}]}

In [21]:
import pprint
for disease_entity in disease_entities_extracted:
    result = disease_normalizer.normalize(disease_entity["entity"])
    pprint.pp(result)

Loaded dictionary from cached file ./tmp/cached_biosyn-biobert-ncbi-disease_dict_Disease_20210630.pk
{'mention': 'nephrogenic diabetes insipidus ndi',
 'predictions': [{'name': 'diabetes insipidus, nephrogenic, x-linked|congenital nephrogenic diabetes insipidus|diabetes insipidus, nephrogenic, type ii|vasopressin-resistant diabetes insipidus|nephrogenic diabetes insipidus, type is|diabetes insipidus, nephrogenic, autosomal|acquired nephrogenic diabetes insipiduses|congenital nephrogenic diabetes insipiduses|acquired nephrogenic diabetes insipidus|diabetes insipidus renalis|adh-resistant diabetes insipidus|nephrogenic diabetes insipidus|adh-resistant diabetes insipiduses|diabetes insipidus, nephrogenics|diabetes insipidus, nephrogenic, type 1|diabetes insipidus, nephrogenic|diabetes insipidus, nephrogenic, type i|nephrogenic diabetes insipidus, type iis|nephrogenic diabetes insipidus, type i|nephrogenic diabetes insipidus, type ii|vasopressin resistant diabetes insipidus|Familial nephro

In [6]:
confusion_matrix = {"test":[0,0,0]}
categories = []

def match_entity(entity_a, entity_b):
    return entity_a['category'] == entity_b['category'] and\
model.query(entity_a['entity'], top_k=1)[2] == model.query(entity_b['entity'], top_k=1)[2]

def macro_avg_f1_score(ans, gt):
    pass
    # for sample_ans, sample_gt in zip(ans, gt):
        # for category in categories:
            # get all entities in the category
    # ans_entities = [entity['entity']
    #                 for entity in sample_ans if entity['category'] == category]
    # ans_entities = [entity['entity'] for entity in ans]
    # # gt_entities = [entity['text']
    # #                for entity in sample_gt if entity['category'] == category]
    # gt_entities = [entity['text'][0] for entity in gt]
    # TP = len(set(ans_entities) & set(gt_entities))
    # FP = len(set(ans_entities) - set(gt_entities))
    # FN = len(set(gt_entities) - set(ans_entities))
    # confusion_matrix["test"][0] += TP
    # confusion_matrix["test"][1] += FP
    # confusion_matrix["test"][2] += FN
    # f1_score = []
    # # for category in categories:
    # TP, FP, FN = confusion_matrix["test"]
    # precision = TP/(TP+FP) if TP+FP else float('nan')
    # recall = TP/(TP+FN) if TP+FN else float('nan')
    # f1_score.append(2*(precision * recall) / (precision + recall))
    # return sum(f1_score) / len(f1_score)

macro_avg_f1_score(entities_extracted, entities)

NameError: name 'entities_extracted' is not defined